In [1]:
# pip install langchain
#%pip install -qU langchain-groq
# %pip install pandas
#install Langchain community package
# %pip install -qU langchain_community beautifulsoup4

In [2]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    groq_api_key = os.getenv("API_KEY")
)

In [3]:
response = llm.invoke("What are some other methods in the ChatGroq other than temperature, n_results? Don't include any asterisk symbol in the answer.") 

In [4]:
print(response.content)

Here are some other methods in the ChatGroq model:

1. max_tokens: This parameter controls the maximum number of tokens in the generated response.
2. top_p: This parameter controls the diversity of the generated response by specifying the top p tokens to consider at each step.
3. presence_penalty: This parameter penalizes the model for repeating the same token multiple times in the generated response.
4. frequency_penalty: This parameter penalizes the model for generating tokens that have already been generated in the response.
5. stop: This parameter specifies a sequence of tokens that, if generated, will stop the generation process.
6. eos_token: This parameter specifies the end-of-sequence token that will be appended to the generated response.
7. pad_token: This parameter specifies the padding token that will be used to pad the input sequence to the maximum length.
8. length_penalty: This parameter controls the penalty for generating longer responses.
9. no_repeat_ngram_size: This p

In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.ibm.com/job/20957333/software-developer-ahmedabad-in/")

page_data = loader.load().pop().page_content

# print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Make a list of all your dependencies so that they are easily available to others.

In [6]:
# pip freeze > requirements.txt

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """"
        ###Scraped text from the website
        {page_data}
        ###Instruction
        The scraped text of from the career's page of a website.
        You job is to extract the job postings and return them in json format containing following keys:
        `role`, `experience`, `skills`, and `description`.
        ### VALID JSON (NO PREAMBLE)
    """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input= {'page_data': page_data})
print(res.content) #This res object is of type string

Here is the extracted job posting in JSON format:

```
{
  "role": "Software Developer",
  "experience": "Entry Level",
  "skills": [
    "Java/C/C++/Go/Scala",
    "SQL databases (Db2, PostgreSQL, MySQL, Oracle, SQL Server)",
    "Virtualization and Containerization technologies",
    "Docker and Kubernetes frameworks",
    "Cloud Services (IBM Cloud, Amazon Web Services, Microsoft Azure)",
    "Linux/UNIX Operating Systems",
    "Bash/Perl/Python/Ruby (optional)"
  ],
  "description": "Design, develop, test, operate and maintain database features in our products and services and tools to provide a secure environment for the product to be used by customers in the cloud. Evaluate new technologies and processes that enhance our service capabilities. Documenting and sharing your experience, mentoring others."
}
```

Let me know if you need any further assistance!


Converting string 'res' to a JSON object

In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)

json_res

{'role': 'Software Developer',
 'experience': 'Entry Level',
 'skills': ['Java/C/C++/Go/Scala',
  'SQL databases (Db2, PostgreSQL, MySQL, Oracle, SQL Server)',
  'Virtualization and Containerization technologies',
  'Docker and Kubernetes frameworks',
  'Cloud Services (IBM Cloud, Amazon Web Services, Microsoft Azure)',
  'Linux/UNIX Operating Systems',
  'Bash/Perl/Python/Ruby (optional)'],
 'description': 'Design, develop, test, operate and maintain database features in our products and services and tools to provide a secure environment for the product to be used by customers in the cloud. Evaluate new technologies and processes that enhance our service capabilities. Documenting and sharing your experience, mentoring others.'}

In [9]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


Creating persistent client will store our records inside the vectorstore folder

In [10]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [11]:
links = collection.query(query_texts=["Experience in python", 'Experience in React'],n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [12]:
job = json_res
job['skills']

['Java/C/C++/Go/Scala',
 'SQL databases (Db2, PostgreSQL, MySQL, Oracle, SQL Server)',
 'Virtualization and Containerization technologies',
 'Docker and Kubernetes frameworks',
 'Cloud Services (IBM Cloud, Amazon Web Services, Microsoft Azure)',
 'Linux/UNIX Operating Systems',
 'Bash/Perl/Python/Ruby (optional)']

In [13]:
links = collection.query(query_texts=job['skills'],n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [14]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Atharva, a business development executive at Innovex Solutions, an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools. 
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
    process optimization, cost reduction, and heightened overall efficiency. 
    Your job is to write a cold email to the client regarding the job mentioned above, describing the capability of Innovex Solutions 
    in fulfilling their needs.
    Also, add the most relevant ones from the following links to showcase Innovex Solutions' portfolio: {link_list}
    Remember you are Atharva, BDE at Innovex Solutions. 
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)


Subject: Expert Software Development Services for Your Cloud-Based Product

Dear [Client],

I came across your job posting for a Software Developer and was impressed by the scope of the project. As a Business Development Executive at Innovex Solutions, I believe our team can provide the expertise you need to design, develop, test, operate, and maintain database features for your cloud-based product.

Our team of skilled software developers has extensive experience in Java/C/C++/Go/Scala, SQL databases (Db2, PostgreSQL, MySQL, Oracle, SQL Server), Virtualization and Containerization technologies, Docker and Kubernetes frameworks, Cloud Services (IBM Cloud, Amazon Web Services, Microsoft Azure), and Linux/UNIX Operating Systems. We're confident that our expertise will help you create a secure environment for your product to thrive in the cloud.

At Innovex Solutions, we've successfully delivered numerous projects that showcase our capabilities in software development, DevOps, and cloud s